# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/schema.sql")
if not sql_schema:
    print("Warning: Could not load schema.sql. Lab may not function correctly.")

2025-10-29 15:01:30,844 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [2]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""You are a senior Python developer building a FastAPI backend for an Employee Onboarding Tool. Your task is to generate a complete, production-ready FastAPI application with in-memory data storage.

## PROJECT CONTEXT
This is a prototype phase where we validate API structure before integrating with a live database. The application will eventually connect to a SQLite database, but for now, use a simple in-memory Python list as the data store.

## DATABASE SCHEMA REFERENCE
Here is the SQL schema that defines our data structure:

{sql_schema}

## REQUIREMENTS
Generate a single, complete Python file (`main_in_memory.py`) that includes:

### 1. PYDANTIC MODELS
Create Pydantic models for data validation:
- **UserCreate**: For POST requests (fields that users provide when creating a user)
- **UserUpdate**: For PUT requests (fields that can be updated, all optional)
- **UserResponse**: For API responses (includes the auto-generated `id` field)

Follow these conventions:
- Use appropriate field types based on the SQL schema
- Make fields optional where appropriate (use `Optional[type]`)
- Include field validation (e.g., email format, string lengths)
- Add helpful docstrings and examples using Pydantic's `Field()` with descriptions

### 2. IN-MEMORY DATABASE
Create a simple in-memory data store:
- Use a Python list named `fake_db` to store user dictionaries
- Initialize with 2-3 sample users for testing purposes
- Include a counter variable for generating unique IDs

### 3. FASTAPI CRUD ENDPOINTS
Implement complete CRUD operations for the `users` resource:

**CREATE - POST /users/**
- Accept a `UserCreate` model
- Generate a new unique ID
- Add the user to `fake_db`
- Return a `UserResponse` with status code 201

**READ - GET /users/**
- Return a list of all users
- Response should be `List[UserResponse]`

**READ - GET /users/{{user_id}}**
- Accept `user_id` as a path parameter
- Return a single user if found
- Raise HTTPException 404 if user not found

**UPDATE - PUT /users/{{user_id}}**
- Accept `user_id` as path parameter and `UserUpdate` model
- Update only the fields that are provided (partial updates)
- Return the updated `UserResponse`
- Raise HTTPException 404 if user not found

**DELETE - DELETE /users/{{user_id}}**
- Accept `user_id` as path parameter
- Remove user from `fake_db`
- Return a success message dictionary
- Raise HTTPException 404 if user not found

## CODE QUALITY REQUIREMENTS
- Write clean, well-commented code with docstrings
- Include proper error handling with appropriate HTTP status codes
- Use type hints throughout
- Follow PEP 8 style guidelines
- Add a root endpoint (GET /) that returns a welcome message
- Include proper FastAPI imports and app initialization

## OUTPUT FORMAT
Provide ONLY the complete Python code for `main_in_memory.py`. Do not include:
- Explanatory text before or after the code
- Installation instructions
- Multiple file suggestions
- Markdown formatting around the code

The output should be a single, ready-to-run Python file that can be executed with:
```
uvicorn main_in_memory:app --reload
```

Generate the complete code now:"""

print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

--- Generating FastAPI app with in-memory database ---
# main_in_memory.py
# A complete, production-ready FastAPI application with in-memory data storage.

from __future__ import annotations

import datetime
from enum import Enum
from typing import List, Optional, Dict

from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, EmailStr, Field, ConfigDict

# --- 1. PYDANTIC MODELS & ENUMS ---

class UserRole(str, Enum):
    """Enumeration for user roles based on the database schema."""
    NEW_HIRE = "new_hire"
    MANAGER = "manager"
    HR_ADMIN = "hr_admin"


class UserBase(BaseModel):
    """Base model for user data, containing common fields."""
    full_name: str = Field(
        ...,
        min_length=3,
        max_length=100,
        description="The full name of the user.",
        examples=["John Doe"]
    )
    email: EmailStr = Field(
        ...,
        description="The user's unique email address.",
        examples=["john.doe@example.com"]
    )

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [3]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""You are a senior Python developer working on a FastAPI backend application that uses SQLAlchemy for database access. Your task is to generate the database layer code that will connect to a live SQLite database.

## PROJECT CONTEXT
You are building the database integration layer for an Employee Onboarding Tool. The SQLite database file (`onboarding.db`) already exists and was created on Day 2 of the project. Now you need to generate the SQLAlchemy ORM models and session management code that will be manually integrated into the FastAPI application.

## DATABASE SCHEMA REFERENCE
Here is the SQL schema that defines the existing database structure:

{sql_schema}

## REQUIREMENTS
Generate TWO separate, distinct code blocks that will be manually integrated:

---

### CODE BLOCK 1: SQLAlchemy ORM Models

Generate SQLAlchemy model classes that map to the database tables defined in the schema above.

**Requirements:**
- Use SQLAlchemy's declarative base approach
- Import necessary components from `sqlalchemy` and `sqlalchemy.orm`
- Create a `Base` declarative base class
- Define a `User` model class that:
  - Inherits from `Base`
  - Sets `__tablename__` to match the SQL schema
  - Maps all columns from the schema with appropriate SQLAlchemy types
  - Includes proper type hints
  - Uses `mapped_column` for column definitions (SQLAlchemy 2.0 style preferred)
  - Sets primary key, nullable constraints, and default values as per the schema
- Include comprehensive docstrings and inline comments
- Follow SQLAlchemy 2.0+ best practices

---

### CODE BLOCK 2: Database Session Management

Generate the database engine, session factory, and FastAPI dependency function.

**Requirements:**
- Import necessary components from `sqlalchemy`, `sqlalchemy.orm`, and `typing`
- Define `SQLALCHEMY_DATABASE_URL` pointing to: `sqlite:///./artifacts/onboarding.db`
- Create the database engine with:
  - Proper `connect_args` for SQLite (check_same_thread=False)
  - Connection pooling configuration if appropriate
- Create a `SessionLocal` session factory using `sessionmaker`
- Define a `get_db()` generator function that:
  - Yields a database session
  - Follows FastAPI's dependency injection pattern
  - Uses try/finally to ensure proper session cleanup
  - Includes proper type hints (Generator[Session, None, None])
- Include comprehensive docstrings explaining the purpose of each component
- Add comments explaining SQLite-specific configurations

---

## CODE QUALITY REQUIREMENTS
- Use clear, descriptive variable names
- Include detailed docstrings for classes and functions
- Add inline comments explaining SQLite-specific configurations
- Use type hints throughout
- Follow PEP 8 style guidelines
- Ensure code is ready for manual integration (no placeholder imports)

## OUTPUT FORMAT
Provide TWO clearly separated code blocks:

1. First code block: SQLAlchemy ORM Models (preceded by a comment header)
2. Second code block: Database Session Management (preceded by a comment header)

**IMPORTANT:** 
- Clearly label each code block with a comment header
- Do NOT combine them into a single file
- Do NOT include explanatory prose between the code blocks
- Do NOT include markdown formatting
- The code blocks should be copy-paste ready for manual integration

Generate the two code blocks now:"""

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

--- Generating SQLAlchemy Models and Session Code ---

--- Generated Database Code ---
```python
# CODE BLOCK 1: SQLAlchemy ORM Models
# File: models.py
# Description: This file contains the SQLAlchemy ORM models that map to the
#              Employee Onboarding Tool's database schema.

from __future__ import annotations

import datetime
from typing import List, Optional

from sqlalchemy import (
    Boolean,
    Column,
    Date,
    DateTime,
    ForeignKey,
    Integer,
    PrimaryKeyConstraint,
    String,
    Table,
    Text,
    func,
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship


class Base(DeclarativeBase):
    """Base class for all SQLAlchemy ORM models."""
    pass


# Association table for the many-to-many relationship between OnboardingPlan and Template
plan_templates_table = Table(
    "plan_templates",
    Base.metadata,
    Column("plan_id", Integer, ForeignKey("onboarding_plans.plan_id", ondelete="CASCADE"), primary_key=True),
    C

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.